In [5]:
# 라이브러리 정의
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [6]:
# 모델 생성
# from_pretrained(Bert모델 이름)
# klue/bert-base : 버트 모델 이름으로 [mask] 단어를 맞추기 위한 모델링 구조로 로드
# 함께 사용되는 라이브러리는 AutoTokenizer의 from_pretrained() 함수
model = TFBertForMaskedLM.from_pretrained("klue/bert-base",
                                         from_pt=True)
# from_pretrained("model이 설정한 Bert모델 이름과 동일")
# 모델이 학습 되었을 당시에 사용되었던 토큰나이저를 자동으로 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
print(model)
print(tokenizer)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


BertTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [9]:
# 데이터 정의
text = "축구는 정말 재미있는 [MASK]다."
print(text)

축구는 정말 재미있는 [MASK]다.


In [10]:
# 마스크 언어 모델 입력으로 넣기
# 결과 값은 텐서
inputs = tokenizer(text, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[   2, 4713, 2259, 3944, 6001, 2259,    4,  809,   18,    3]])>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])>}


In [11]:
### 정수 인코딩 결과 확인하기
print(inputs["input_ids"])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


In [13]:
### 문장 단위를 세그먼트라는 용어를 사용
# - 문장이 1개 있으면 세그먼트 0
# - 세그먼트 확인하기
print(inputs["token_type_ids"])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


In [12]:
from transformers import FillMaskPipeline

# 모델 훈련시키기
# 훈련된 모델 연결
# model과 tokenizer 연결
# 연결은 FillMaskPipline 라이브러리에 의해 연결
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)
print(pip)

In [14]:
# MASK 예측 
pip("축구는 정말 재미있는 [MASK]다.")

[{'score': 0.8963512182235718,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.025957640260457993,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033938102424145,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924390956759453,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844238542020321,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

In [15]:
pip("나는 공부가 [MASK]다. 그래도 해야 [MASK]다.")

[[{'score': 0.13137081265449524,
   'token': 1330,
   'token_str': '싫',
   'sequence': '[CLS] 나는 공부가 싫 다. 그래도 해야 [MASK] 다. [SEP]'},
  {'score': 0.08335502445697784,
   'token': 1415,
   'token_str': '없',
   'sequence': '[CLS] 나는 공부가 없 다. 그래도 해야 [MASK] 다. [SEP]'},
  {'score': 0.08089631795883179,
   'token': 4390,
   'token_str': '힘들',
   'sequence': '[CLS] 나는 공부가 힘들 다. 그래도 해야 [MASK] 다. [SEP]'},
  {'score': 0.0550154447555542,
   'token': 1560,
   'token_str': '좋',
   'sequence': '[CLS] 나는 공부가 좋 다. 그래도 해야 [MASK] 다. [SEP]'},
  {'score': 0.05151516571640968,
   'token': 4258,
   'token_str': '어렵',
   'sequence': '[CLS] 나는 공부가 어렵 다. 그래도 해야 [MASK] 다. [SEP]'}],
 [{'score': 0.11688510328531265,
   'token': 582,
   'token_str': '겠',
   'sequence': '[CLS] 나는 공부가 [MASK] 다. 그래도 해야 겠 다. [SEP]'},
  {'score': 0.1035318523645401,
   'token': 1902,
   'token_str': '했',
   'sequence': '[CLS] 나는 공부가 [MASK] 다. 그래도 해야 했 다. [SEP]'},
  {'score': 0.09695763885974884,
   'token': 1891,
   'token_str': '한',
  

In [16]:
pip("어벤져스는 정말 재미있는 [MASK]다.")

[{'score': 0.8382407426834106,
  'token': 3771,
  'token_str': '영화',
  'sequence': '어벤져스는 정말 재미있는 영화 다.'},
 {'score': 0.02827559970319271,
  'token': 568,
  'token_str': '거',
  'sequence': '어벤져스는 정말 재미있는 거 다.'},
 {'score': 0.017189383506774902,
  'token': 4665,
  'token_str': '드라마',
  'sequence': '어벤져스는 정말 재미있는 드라마 다.'},
 {'score': 0.014989701099693775,
  'token': 3758,
  'token_str': '이야기',
  'sequence': '어벤져스는 정말 재미있는 이야기 다.'},
 {'score': 0.009382650256156921,
  'token': 4938,
  'token_str': '장소',
  'sequence': '어벤져스는 정말 재미있는 장소 다.'}]